In [ ]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    """提取PDF中的文本，自动判断单列或两列布局，输出纯文本字符"""
    doc = fitz.open(pdf_path)
    pages_text = []

    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)

        # 获取页面的尺寸
        page_width = page.rect.width
        page_height = page.rect.height

        # 获取页面的文本块，用于判断布局
        blocks = page.get_text("blocks")
        left_count, right_count = 0, 0

        for block in blocks:
            # 计算文本块的中心位置
            block_center_x = (block[0] + block[2]) / 2
            if block_center_x < page_width / 2:
                left_count += 1
            else:
                right_count += 1

        # 如果右侧有大量文本块，则判断为两列布局
        is_two_columns = right_count > left_count * 0.3  # 阈值可调整

        if is_two_columns:
            # 按两列布局处理文本
            left_rect = fitz.Rect(0, 0, page_width / 2, page_height)  # 左侧区域
            right_rect = fitz.Rect(page_width / 2, 0, page_width, page_height)  # 右侧区域

            # 提取左列和右列的文本并移除多余的换行符
            left_text = page.get_text("text", clip=left_rect).replace('\n', ' ')
            right_text = page.get_text("text", clip=right_rect).replace('\n', ' ')

            # 合并左列和右列文本，形成连续的段落
            page_text = left_text + " " + right_text
        else:
            # 按单列布局处理文本，移除换行符
            page_text = page.get_text("text").replace('\n', ' ')

        # 将处理后的文本添加到总文本中
        pages_text.append(page_text)

    return pages_text  # 保持每个页面的文本独立